In [1]:
import pandas as pd
import numpy as np
from scipy import integrate
import seaborn as sns
import matplotlib.pyplot as mp
import plotly.offline as pyo
import plotly.graph_objs as go;
import pandas_datareader as datareader
import datetime as dt
import math
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')


### Problem Statement 4:
 You are provided with the '50_Startups' data. Using the given features, you must predict the profit of these startups.
 Dataset Description:
 
I.R&D Spend: Expenditures in Research and Development
 
II.Administration: Expenditures in Administration
 
III.Marketing Spend: Expenditures in Marketing
 
IV.State: In which state the company belongs to
 
V.Profit: The profit made by the company
 
Write a Python code to perform the following tasks mentioned:
 
1.Load the data, check its shape and check for null values
 
2.Convert categorical features to numerical values using Label Encoder
 
3.Split the dataset for training and testing 
 
 4.Train the model using sklearn(linear regression), also find the intercept and coefficient from the trained model
 
5.Predict the profits of test data and evaluate the model using r2 score and mean squared error

6.Regularize the model using Ridge Regression and find the Score 
  
7.Regularize the model using LassoRegression and find the Score

In [16]:
data=pd.read_csv("D:\Shivam\study material\CSV\\50_Startups.csv")
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [22]:
data.describe()

,R&D Spend,Administration,Marketing Spend,Profit
count,50.000000,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800,112012.639200
std,45902.256482,28017.802755,122290.310726,40306.180338
min,0.000000,51283.140000,0.000000,14681.400000
25%,39936.370000,103730.875000,129300.132500,90138.902500
50%,73051.080000,122699.795000,212716.240000,107978.190000
75%,101602.800000,144842.180000,299469.085000,139765.977500
max,165349.200000,182645.560000,471784.100000,192261.830000


In [23]:
data.isnull().sum()

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

dataset has no null values